In [5]:
from matplotlib.image import imread
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
from skimage import transform
import cv2 as cv
import numpy as np


In [7]:
directory = "../Dataimage/Train"

In [8]:
def list_files(directory):
    file_list = []
    # Iterate through all files in the directory and its subdirectories
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Construct the full path to the file
            file_path = os.path.join(root, file)
            image = cv.imread(file_path)
            height, width, _ = image.shape
            image = cv.resize(image,(800,800))
            image = normalize_image(image)
            image = sharpen_image_with_laplacian(image)
            cv.imwrite(f"../Dataimage/Result_2/Train/{file}", image)


In [9]:

def normalize_image(image):
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    normalized_image = cv.normalize(gray_image, None, 0, 255, cv.NORM_MINMAX, dtype=cv.CV_8U)
    mean_intensity = np.mean(normalized_image)

    if mean_intensity >= 200:
        adjusted_image = np.where(normalized_image > 100, normalized_image - 50, normalized_image).astype(np.uint8)
    elif mean_intensity < 100:
        adjusted_image = np.where(normalized_image < 155, normalized_image + 50, normalized_image).astype(np.uint8)
    else:
        adjusted_image = normalized_image

    return adjusted_image



In [10]:

def sharpen_image_with_laplacian(image):
    # Apply the Laplacian operator to detect edges\
    color_image = cv.cvtColor(image, cv.COLOR_GRAY2BGR)
    laplacian = cv.Laplacian(image, cv.CV_64F)

    # Calculate the sharpened image by adding the Laplacian image to the original image
    # Normalize the Laplacian result to the range [0, 255]
    laplacian_norm = cv.normalize(laplacian, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)

    # Convert Laplacian result back to 3-channel format
    laplacian_rgb = cv.merge((laplacian_norm, laplacian_norm, laplacian_norm))

    # Calculate the sharpened image by adding the Laplacian-enhanced image to the original image
    sharpened_image = cv.addWeighted(color_image, 1.5, laplacian_rgb, -0.5, 0)

    return sharpened_image


list_files("../Dataimage/Train")